In [1]:
import pandas as pd
import bagpy
import pickle
import pathlib

/tmp/ipykernel_755616/1222988256.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Convert ROS bag file to pickle

In [2]:
def extract_data(in_file: str, out_file: str) -> pd.DataFrame:
    bag = bagpy.bagreader(in_file)
    topic_data = []
    for topic in bag.topics:
        df = pd.read_csv(bag.message_by_topic(topic))
        topic_data.append(df[df.columns.drop(list(df.filter(regex="head")))])

    with open(out_file, "wb") as handle:
        pickle.dump(topic_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


bag_path_col_pair = pathlib.Path().resolve().parent / "bags"
pkl_path = pathlib.Path().resolve().parent / "pickle"
pathlib.Path(pkl_path).mkdir(parents=True, exist_ok=True)
for file in bag_path_col_pair.glob("*.bag"):
    extract_data(file.as_posix(), pkl_path / file.with_suffix(".pkl").name)

[INFO]  Data folder /home/krzysiek/Documents/Programming/laas/post-treatment-panda-torque-mpc/bags/sinu_motion_around_z_weight_2023-09-12-16-44-18 already exists. Not creating.


In [220]:
pkl_path = pathlib.Path().resolve().parent / "pickle"
b = {}
for file in pkl_path.glob("*.pkl"):
    with open(file, "rb") as handle:
        b[file.name] = pickle.load(handle)